In [1]:
import pandas as pd
from cleaning import create_demand
import pyomo.environ as pyo
import pyomo

# Data import

In [2]:
nodes = pd.read_pickle("../data/original/nodes.pkl")
channels = pd.read_pickle("../data/original/channels.pkl")

In [3]:
nodes.to_csv("../data/original/nodes.csv")
channels.to_csv("../data/original/channels.csv")

## Modeling

Change the logic of CHANNELS set construction by considering the pair of nodes as the edge identifier.
In this way I also have to deal with:
- [X] multiedges, aka multiple edges between two nodes. I can not consider those for simplicity
- [X] directed relationship. I need to understand if flipping the channels - as done before - can be reasonable. I also need to determine if the flipped pair of nodes is considered a valid index by pyomo.

In [4]:
## Note that the following are arbitrary policies, that can be changed as needed.
## Deal with multi-edges (aka multiple channels between two peers):
## - average rate fee
## - average base fee,
## - average capacity
## - keep one of the two channel ids
aggregation_dict = {
    "channel_id": "first",
    "rate_fee": "mean",
    "base_fee": "mean",
    "capacity": "sum"
}
channels.reset_index(inplace=True)
channels = channels.groupby(["node1_pub", "node2_pub"]).agg(aggregation_dict)
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

In [5]:
## To delete from [outgoing] and [incoming] lists the multi-edge
for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "outgoing_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "outgoing_channels"] = chan

for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "incoming_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "incoming_channels"] = chan

In [6]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

In [7]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals)

In [8]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [9]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS) + sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

### Constraints

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$


In [10]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * model.x[(a, b)]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{si} - \sum_{(i,t) \in E} amount_{it} = b_i \text{ } \forall i \in V$$


In [11]:
nodes = create_demand(nodes, 130000)

Transaction of 130000 sats from gocharlie-lightning-node-1 to the_holy_cheese_grater.


In [12]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    return sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels']) - sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels']) == nodes.loc[n, "demand"]

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True) 

## Solving the model

In [13]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())


Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpasn85wb3.pyomo.lp -stat=1 -solve -solu /tmp/tmpasn85wb3.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 32351 (-20513) rows, 73416 (-20580) columns and 141995 (-45617) elements
Statistics for presolved model
Original problem has 46998 integers (46998 of which binary)
Presolved problem has 28909 integers (28909 of which binary)
==== 3511 zero objective 3006 different
==== absolute objective values 3006 different
==== for integers 0 zero objective 750 different
==== for integers absolute objective values 750 different
===== end objective counts


Problem has 32351 rows, 73416 columns (69905 with objective) and 141995 elements
There are 29586 singletons with objective 
Column breakdown:
26660 of type 0.0->inf, 17847 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0

In [14]:
ris = []
for i,v in model.a.extract_values().items():
    if v != 0:
        ris.append((i, v))

print(ris)

[(('026a38b5d58e84d88b1e4ef072153daeb72c317468d33878978a3728fe83d59ecf', '029efe15ef5f0fcc2fdd6b910405e78056b28c9b64e1feff5f13b8dce307e67cad'), 130000.0), (('02881003525adb05fef7f73eacc8f31903791ced708a4fc2680de08acae949006a', '02cfdc6b60e5931d174a342b20b50d6a2a17c6e4ef8e077ea54069a3541ad50eb0'), 130000.0), (('02a14c5c4fedba011d5a822f44bc4cd968484b94d92810872971ce498f278e34f1', '029efe15ef5f0fcc2fdd6b910405e78056b28c9b64e1feff5f13b8dce307e67cad'), 130000.0), (('02cfdc6b60e5931d174a342b20b50d6a2a17c6e4ef8e077ea54069a3541ad50eb0', '026a38b5d58e84d88b1e4ef072153daeb72c317468d33878978a3728fe83d59ecf'), 130000.0), (('0364913d18a19c671bb36dd04d6ad5be0fe8f2894314c36a9db3f03c2d414907e1', '02cca6c5c966fcf61d121e3a70e03a1cd9eeeea024b26ea666ce974d43b242e636'), 130000.0), (('0364913d18a19c671bb36dd04d6ad5be0fe8f2894314c36a9db3f03c2d414907e1', '039174f846626c6053ba80f5443d0db33da384f1dde135bf7080ba1eec465019c3'), 130000.0), (('03c5528c628681aa17ab9e117aa3ee6f06c750dfb17df758ecabcd68f1567ad8c1', '03

In [31]:
DF = pd.DataFrame()
for v in model.component_objects(pyo.Var,active=True):
    for index in v:
        DF.loc[index, v.name] = pyo.value(v[index])

KeyError: "None of [Index(['0200000000727d3b67513c916f16975e3bf8f3304cf3fcf0ed855e2ae41888f461', '0360a41eb8c3fe09782ef6c984acbb003b0e1ebc4fe10ae01bab0e80d76618c8f4'], dtype='object')] are in the [index]"

In [30]:
DF

""
